In [1]:
import pandas as pd
import pickle
import numpy as np
from Constants import Const
import torch
import torch.nn as nn
import cv2
import glob
from FileUtils import *

In [2]:
import sparseconvnet as scn
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
files = glob.glob(Const.pointcloud_dir+'pclouds_*.json')
all_files = []
max_num = 10
for file in files:
    with open(file,'r') as f:
        test = simplejson.load(f)
    if int(test['patient_id']) not in Const.bad_ids:
        all_files.append(test)
    if len(all_files) > max_num:
        break
len(all_files)

11

In [45]:

def downsample_pointcloud(pcloud,k=1,max_points = 5000):
    pc = o3d.geometry.PointCloud()
    pc.points = o3d.utility.Vector3dVector(pcloud['coordinates'])
    #save dose as rgb color for when they're discretized
    pc.colors = o3d.utility.Vector3dVector(np.stack([pcloud['dose_values'] for c in range(3)],axis=-1))
    pc = pc.voxel_down_sample(k)#to make it uniform for the convolution
    if len(pc.points) > max_points:
        pc = pc.farthest_point_down_sample(max_points)
    points = np.asarray(pc.points)
    colors = np.asarray(pc.colors)[:,0]
    return {'coordinates': points,'dose_values': colors}

def compile_pointclouds(patient,batch_index=None, organs = Const.organ_list + ['gtv','gtvn'],
                        downsample_k=1,max_points=2000):
    cpc = patient['contour_pointclouds']
    all_points = []
    all_values = []
    for organ, entry in cpc.items():
        if organs is not None and organ not in organs:
            continue
        coords = entry['coordinates']
        values = entry['dose_values']
        all_points.append(np.array(coords))
        all_values.append(np.array(values).reshape(-1,1))
    all_points = np.vstack(all_points)
    all_values = np.vstack(all_values).ravel()
    entry = {'coordinates': all_points,'dose_values':all_values}
    if downsample_k > 0:
        entry = downsample_pointcloud(entry,downsample_k,max_points)
    if batch_index is not None:
        ap = entry['coordinates']
        bi = np.full((ap.shape[0],1),batch_index)
        entry['coordinates'] =np.hstack((ap,bi))
    entry['patient_id'] = patient['patient_id']
    return entry


In [46]:
all_patients= [compile_pointclouds(p,batch_index=i) for i,p in enumerate(all_files)]
for p in all_patients:
    print(p['coordinates'].shape)

(2000, 4)
(2000, 4)
(2000, 4)
(1625, 4)
(2000, 4)
(2000, 4)
(2000, 4)
(2000, 4)
(2000, 4)
(2000, 4)
(2000, 4)


In [39]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [51]:
def torchify(locs,features,device=None):
    if device is None:
        device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    locs[:,0:3] = locs[:,0:3]*100
    locs = torch.LongTensor(locs.astype(int))
    features = torch.FloatTensor(features.reshape(-1,1))
    return locs.to(device), features.to(device)

def compile_patient_batch(patients):
    patients = [compile_pointclouds(p,batch_index=i) for i,p in enumerate(all_files)]
    all_locations = []
    all_features = []
    for p in patients:
        all_locations.append(p['coordinates'])
        all_features.append(p['dose_values'].reshape(-1,1))
    all_locations = np.concatenate(all_locations,axis=0)
    all_features = np.concatenate(all_features)
    print(all_locations.shape)
    return torchify(all_locations,all_features)

test_l, test_f = compile_patient_batch(all_files[0:4])
test_l.shape

(21625, 4)


torch.Size([21625, 4])

In [54]:
class SparseCNN(nn.Module):
    
    def __init__(self,dims=3,features=1,
                 output_dim = 1,
                 manifold_m = [32,32],
                 manifold_size=[3,3],
                 full_m = [32],
                 full_size = 5,
                 reps=1,
                ):
        nn.Module.__init__(self)
        self.dims = dims
        self.features = features
    
        self.input_layer = scn.InputLayer(dims,features) 
        self.batchNorm = scn.BatchNormalization(features) # I don't actually know what input planes means
        self.manifold = scn.SubmanifoldConvolution(dims,features,manifold_m[0],manifold_size[0],True)
        self.manifold2 = scn.SubmanifoldConvolution(dims,manifold_m[0],manifold_m[1],manifold_size[1],True)

        self.batchNorm2 = scn.BatchNormReLU(manifold_m[-1])
        self.fullConv = scn.FullConvolution(dims,manifold_m[-1],full_m[0],full_size,1,True)
        
        #with submanifold it will be dims*last # of features, fullConvolution seems to be dims*#features*(full_size**3)
        self.final_dim = full_m[-1]*(full_size**3)*dims
        self.sparseToDense = scn.SparseToDense(dims,full_m[-1])

        self.linear = nn.LazyLinear(output_dim)
#         self.shapeContext = scn.ShapeContext(1,features)
        
    def forward(self,locs,features,batch_size):
        x = self.input_layer((locs,features,3))
        x = self.batchNorm(x)
        x = self.manifold(x)
        x = self.manifold2(x)
    
        x = self.batchNorm2(x)
        x = self.fullConv(x)
        x = self.sparseToDense(x)
        x = x.view(batch_size,-1)
        x = self.linear(x)
        return x
    
model = SparseCNN()
model = model.to(device)
model(test_l,test_f,3)

RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
model